# Retail Store Sales Forecasting
## Enhanced: SARIMA Model, Hyperparameter Tuning, and External Regressors

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import TimeSeriesSplit, ParameterGrid
from sklearn.metrics import mean_squared_error


In [ ]:
# Data loading (update path as needed)
df = pd.read_csv("D:/internship/Retail_Store_Sales_Forecasting/data/retail_sales_mock_data.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date').sort_index()

# Resample to monthly (if not already)
df_monthly = df.resample('M').sum()


### Handling Outliers and Interpolating

In [ ]:
from scipy.stats import zscore
z_scores = zscore(df_monthly)
outliers = np.abs(z_scores) > 3
df_monthly[outliers] = np.nan
df_monthly = df_monthly.interpolate()


### Add (Example) Economic Indicators as Regressors
*Assume your dataset does not already have economic indicators. This is a synthetic addition, in practice you would join real economic indicators.*

In [ ]:
# Synthetic economic indicator (e.g., monthly CPI or unemployment rate, here random)
np.random.seed(42)
df_monthly['EconIndicator'] = np.random.normal(100, 5, len(df_monthly))

# Preview
display(df_monthly.head())

## Stationarity Check and Differencing

In [ ]:
adf_result = adfuller(df_monthly['SalesAmount'])
print(f"ADF Statistic: {adf_result[0]:.4f}, p-value: {adf_result[1]:.4f}")
if adf_result[1] > 0.05:
    df_monthly['Sales_diff1'] = df_monthly['SalesAmount'].diff()
    df_model = df_monthly.dropna(subset=['Sales_diff1'])
    target_col = 'Sales_diff1'
else:
    df_model = df_monthly.copy()
    target_col = 'SalesAmount'


## SARIMA with External Regressors
Columns used as exogenous variables: Promotion, HolidayMonth, EconIndicator

In [ ]:
exog_cols = ['Promotion', 'HolidayMonth', 'EconIndicator']
exog = df_model[exog_cols]
y = df_model[target_col]

# Train-test split (last 6 months as test set)
train_size = int(len(df_model) - 6)
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
exog_train, exog_test = exog.iloc[:train_size], exog.iloc[train_size:]


### SARIMA Hyperparameter Search
Grid search (example: limited for speed; expand in production):

In [ ]:
param_grid = {
    'order': [(1,1,1), (2,1,2)],
    'seasonal_order': [(1,1,1,12), (0,1,1,12)]
}

best_score = float('inf')
best_params = None
for params in ParameterGrid(param_grid):
    try:
        model = SARIMAX(y_train, exog=exog_train, order=params['order'], seasonal_order=params['seasonal_order'], enforce_stationarity=False, enforce_invertibility=False)
        model_fit = model.fit(disp=False)
        y_pred = model_fit.forecast(steps=len(y_test), exog=exog_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        if rmse < best_score:
            best_score = rmse
            best_params = params
    except Exception as e:
        print(f"Params {params} failed: {e}")
print(f"Best SARIMA params: {best_params}, RMSE: {best_score:.2f}")

### Fit Final Model

In [ ]:
# Fit best model on all data except test set
model_final = SARIMAX(y_train, exog=exog_train, order=best_params['order'], seasonal_order=best_params['seasonal_order'], enforce_stationarity=False, enforce_invertibility=False)
model_fit_final = model_final.fit(disp=False)
y_pred_final = model_fit_final.forecast(steps=len(y_test), exog=exog_test)


### Inverse Differencing (if needed)

In [ ]:
if target_col == 'Sales_diff1':
    # Recover forecasted sales from differenced values
    last_actual = df_monthly['SalesAmount'].iloc[train_size-1]
    y_pred_sales = np.r_[last_actual, y_pred_final].cumsum()[1:]
    y_test_sales = df_monthly['SalesAmount'].iloc[train_size:]
else:
    y_pred_sales = y_pred_final
    y_test_sales = y_test


### Evaluation & Plot

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test_sales, y_pred_sales))
print(f'Final RMSE on test set: {rmse:.2f}')
plt.figure(figsize=(12,6))
plt.plot(df_monthly.index, df_monthly['SalesAmount'], label='Actual')
plt.plot(y_test_sales.index, y_pred_sales, label='SARIMA+Exog Forecast', marker='o')
plt.title('Retail Sales Forecast with SARIMA + External Regressors')
plt.xlabel('Date')
plt.ylabel('Monthly Sales')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

----
**Notes:**
- You can expand the parameter grid for more exhaustive SARIMA tuning.
- Replace the synthetic economic indicator with real data for production.
- You can add more external regressors (weather, special events, etc.) as available.
- Use cross-validation (such as TimeSeriesSplit) for robust model validation in production.